In [14]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
import statsmodels.formula.api as smf

In [15]:
df = pd.read_csv('data/train_data.csv', delimiter=',')

cols = df.columns[3:-1]
numFeatures = len(cols)

df = df.dropna(subset=cols) #drop na cells 
df = df[df.bedrooms <= 15]
df = df[df.sqft_lot <= 1250000]
df = df[df.sqft_above <= 7000]
df = df[df.sqft_basement <= 3500]
df = df[df.sqft_lot15 <= 500000]

scores = []
for i in df['yr_renovated']:
    if i != 0:
        scores.append((i - 2015) + 82)
    else:
        scores.append(0)
df['renovation_score'] = scores

landvalue = []
view = list(df['view'])
water = list(df['waterfront'])
living = list(df['sqft_living'])
lot = list(df['sqft_lot'])
grade = list(df['grade'])
condition = list(df['condition'])
for i in range(len(water)):
    value = (view[i] + 1) * living[i]
    if water[i] == 1:
        value  = value * 2
    value = value / lot[i]
    landvalue.append(value)
df['landvalue'] = landvalue

centerpoint= (47.628591, -122.289796)  #center point of city center
dist_df = df.loc[:,['Unique_idx','lat','long','price']]
dist_df['dist_latLon'] = np.sqrt(np.square(dist_df['lat']- centerpoint[0]) + np.square(dist_df['long']-centerpoint[1]))
df['distFromSeattle'] = dist_df['dist_latLon']

df.head()

,Unique_idx,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,renovation_score,landvalue,distFromSeattle
0,7291,4058200630,20141002T000000,3,1.75,2190,7021,1.0,0,2,...,0,98178,47.5033,-122.232,2180,7155,353000,0,0.935764,0.137979
1,14835,3613600150,20150105T000000,3,2.50,2370,6840,2.0,0,0,...,0,98119,47.6503,-122.366,1590,4400,300523,0,0.346491,0.079236
2,15880,7170200080,20140617T000000,2,1.00,1230,3800,1.0,0,0,...,0,98115,47.6797,-122.292,1610,3800,435000,0,0.323684,0.051157
3,8812,9542300530,20141124T000000,4,2.25,2510,9963,1.0,0,0,...,0,98005,47.5973,-122.177,3110,9963,800000,0,0.251932,0.117056
4,17220,3626039028,20140818T000000,3,1.00,1160,7491,1.0,0,0,...,0,98177,47.7024,-122.359,1800,2267,417500,0,0.154852,0.101178


In [17]:
percent = 0.7  #choose percent to split 
n= df_train2.shape[0] #number of rows in data set before splitting
s= math.floor(percent * n)
y= df_train2.iloc[0:s,-1]
X = df_train2.iloc[0:s,:-1]
X['price']=y  #need to add price as last col for statsmodel

y_test= df_train2.iloc[s:n,-1]
X_test = df_train2.iloc[s:n,:-1]

model = smf.ols('price ~ bedrooms + bathrooms + sqft_living + sqft_lot + floors + landvalue + waterfront + view + condition + grade + sqft_above + sqft_basement + yr_built + zipcode + yr_renovated +renovation_score +distFromSeattle + np.power(distFromSeattle, 2)', X).fit()
#cols2 = ['              bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors','landvalue','waterfront', 'view', 'condition', 'grade', 'sqft_above','sqft_basement', 'yr_built', 'distFromSeattle', 'zipcode', 'lat', 'long', 'yr_renovated', 'renovation_score', 'sqft_living15', 'sqft_lot15', 'price']

model.summary().tables[0]

Dep. Variable:,price,R-squared:,0.744
Model:,OLS,Adj. R-squared:,0.743
Method:,Least Squares,F-statistic:,2060.
Date:,"Wed, 12 Feb 2020",Prob (F-statistic):,0.00
Time:,16:22:37,Log-Likelihood:,-1.6321e+05
No. Observations:,12098,AIC:,3.265e+05
Df Residuals:,12080,BIC:,3.266e+05
Df Model:,17,,
Covariance Type:,nonrobust,,


In [18]:
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,8.072e+07,3.57e+06,22.615,0.000,7.37e+07,8.77e+07
bedrooms,-3.451e+04,2320.638,-14.871,0.000,-3.91e+04,-3e+04
bathrooms,2.154e+04,3835.545,5.615,0.000,1.4e+04,2.91e+04
sqft_living,116.7613,2.546,45.864,0.000,111.771,121.752
sqft_lot,0.2247,0.044,5.138,0.000,0.139,0.310
floors,-4.002e+04,4601.127,-8.698,0.000,-4.9e+04,-3.1e+04
landvalue,1.614e+04,5010.541,3.221,0.001,6317.260,2.6e+04
waterfront,5.31e+05,2.05e+04,25.920,0.000,4.91e+05,5.71e+05
view,4.742e+04,2925.599,16.210,0.000,4.17e+04,5.32e+04
condition,2.955e+04,2733.827,10.808,0.000,2.42e+04,3.49e+04


In [20]:
#compare to test data
yhat = (model.predict(X_test)).astype(int)
delta = yhat - y_test
mse = np.sqrt(np.mean(np.square(delta)))
print('mse (stats model) = ', mse)

mse (stats model) =  188598.00121326977


In [21]:
df = pd.read_csv('data/val_data.csv', delimiter=',')

cols = df.columns[3:-1]
numFeatures = len(cols)

df = df.dropna(subset=cols) #drop na cells 
df = df[df.bedrooms <= 15]
df = df[df.sqft_lot <= 1250000]
df = df[df.sqft_above <= 7000]
df = df[df.sqft_basement <= 3500]
df = df[df.sqft_lot15 <= 500000]

scores = []
for i in df['yr_renovated']:
    if i != 0:
        scores.append((i - 2015) + 82)
    else:
        scores.append(0)
df['renovation_score'] = scores

landvalue = []
view = list(df['view'])
water = list(df['waterfront'])
living = list(df['sqft_living'])
lot = list(df['sqft_lot'])
grade = list(df['grade'])
condition = list(df['condition'])
for i in range(len(water)):
    value = (view[i] + 1) * living[i]
    if water[i] == 1:
        value  = value * 2
    value = value / lot[i]
    landvalue.append(value)
df['landvalue'] = landvalue

centerpoint= (47.628591, -122.289796)  #center point of city center
dist_df = df.loc[:,['Unique_idx','lat','long','price']]
dist_df['dist_latLon'] = np.sqrt(np.square(dist_df['lat']- centerpoint[0]) + np.square(dist_df['long']-centerpoint[1]))
df['distFromSeattle'] = dist_df['dist_latLon']

df.head()

,Unique_idx,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,renovation_score,landvalue,distFromSeattle
0,15544,1310430130,20141009T000000,4,2.75,2790,6600,2.0,0,0,...,2000,0,98058,47.4362,-122.109,2900,6752,0,0.422727,0.264010
1,17454,2540830020,20150401T000000,3,2.25,1630,6449,1.0,0,0,...,1986,0,98011,47.7275,-122.232,1620,7429,0,0.252752,0.114557
2,21548,8835770330,20140819T000000,2,1.50,2370,184231,2.0,0,0,...,2005,0,98045,47.4543,-121.778,3860,151081,0,0.012864,0.540659
3,3427,7732400490,20141105T000000,4,2.50,2270,7665,2.0,0,0,...,1986,0,98052,47.6612,-122.148,2450,8706,0,0.296151,0.145497
4,8809,2800031,20150401T000000,3,1.00,1430,7599,1.5,0,0,...,1930,0,98168,47.4783,-122.265,1290,10320,0,0.188183,0.152323


In [48]:
X = df

yhat = (model.predict(X)).astype(int)

results_df = pd.DataFrame(columns = ['Unique_idx', 'price']) 
results_df['Unique_idx'] = df['Unique_idx']
results_df['price'] = yhat

results_df = results_df.set_index('Unique_idx')
results_df

,price
Unique_idx,
15544,630814
17454,509813
21548,727943
3427,690881
8809,220638
3294,675244
275,580351
8736,690937
6161,602085


In [47]:
results_df.to_csv('results.csv', sep='\t', encoding='utf-8')